In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Luderitz,-26.6481,15.1594,67.73,34,0,6.42,NaN,2022-06-06 21:53:29
1,1,Puerto Ayora,-0.7393,-90.3518,78.76,90,76,5.01,EC,2022-06-06 21:51:36
2,2,Yar-Sale,66.8333,70.8333,43.29,64,34,13.40,RU,2022-06-06 21:53:30
3,3,Saint-Philippe,-21.3585,55.7679,67.59,79,54,8.41,RE,2022-06-06 21:53:30
4,4,Shingu,33.7333,135.9833,67.64,83,19,5.21,JP,2022-06-06 21:53:31


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [7]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Puerto Ayora,-0.7393,-90.3518,78.76,90,76,5.01,EC,2022-06-06 21:51:36
5,5,Oswego,43.4501,-76.1497,78.94,76,100,10.51,US,2022-06-06 21:53:31
17,17,Sangali,7.0694,122.2014,76.73,81,15,0.96,PH,2022-06-06 21:53:35
28,28,Marsh Harbour,26.5412,-77.0636,80.58,79,93,17.00,BS,2022-06-06 21:53:40
30,30,Atuona,-9.8000,-139.0333,78.01,74,15,13.67,PF,2022-06-06 21:53:41
31,31,Alexandria,31.2156,29.9553,75.16,73,0,5.75,EG,2022-06-06 21:53:41
35,35,Phalodi,27.1333,72.3667,85.86,34,0,17.05,IN,2022-06-06 21:53:42
44,44,Poum,-20.2333,164.0167,77.31,69,87,11.95,NC,2022-06-06 21:53:46
47,47,Albany,42.6001,-73.9662,78.64,44,100,5.10,US,2022-06-06 21:51:21
49,49,Hithadhoo,-0.6000,73.0833,81.64,74,94,5.01,MV,2022-06-06 21:51:30


In [13]:
preferred_cities_df.count()

City_ID       22
City          22
Lat           22
Lng           22
Max Temp      22
Humidity      22
Cloudiness    22
Wind Speed    22
Country       22
Date          22
dtype: int64

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,78.76,-0.7393,-90.3518,
5,Oswego,US,78.94,43.4501,-76.1497,
17,Sangali,PH,76.73,7.0694,122.2014,
28,Marsh Harbour,BS,80.58,26.5412,-77.0636,
30,Atuona,PF,78.01,-9.8000,-139.0333,
31,Alexandria,EG,75.16,31.2156,29.9553,
35,Phalodi,IN,85.86,27.1333,72.3667,
44,Poum,NC,77.31,-20.2333,164.0167,
47,Albany,US,78.64,42.6001,-73.9662,
49,Hithadhoo,MV,81.64,-0.6000,73.0833,


In [17]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [18]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [19]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uEDxVP2VA7UI9Xt0Uj60xGfr3nxODmuwcjOrA72EG7Kh7m2fpJAY1l7zoPOK3itoJCUwpI5fIm5fmYnbsFR8vr3ZpDF387xmMesVQP9s1Qm5WcxR4cXLmQVTrTEjwh4Jfzdnmgx9Fgyup_Es_ILbE81ZgoBJ8oR5XrZVTho5MRtm45Tw7h-nKZdQ30H2ABuFTA6WwcgcBvtj3JjkTtUnVhbzYo8XwKL8AItOsycoRDAPizxF4V4EaxnnTPqVi50xjyMKFYTteWiSKe5sgmsiDJDlwv2W4N3hQDA1kEhgD7MJ7K0DFC3-8UUlgX3KsIzzy0oxUUWwPtRojOuB3R-c_3pN9cAYSbajxWjyNjnkycwxma1A8NBd7zZfz1FGTtRStLmOkLQKIBvUiYgW6sXtBoDPb8fcnPCiZ-sE0v_8qDbwX71TtDDmEojB',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [42]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
                print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


In [43]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [46]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [47]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [48]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))